In [3]:
import cv2
import numpy as np

In [4]:
cv2.imshow("", np.random.random([100, 100, 3]))

In [6]:
from midiutil import MIDIFile
from midi2audio import FluidSynth

# Step 1: Create MIDI
midi = MIDIFile(1)
track = 0
time = 0
midi.addTempo(track, time, 120)

channel = 0
volume = 100
duration = 1  # seconds

# Melody notes: C4, D4, E4, G4
notes = [60, 62, 64, 67]

for i, note in enumerate(notes):
    midi.addNote(track, channel, note, time + i * duration, duration, volume)

# Save MIDI file
midi_file = "melody.mid"
with open(midi_file, "wb") as output_file:
    midi.writeFile(output_file)

# Step 2: Convert MIDI to audio
soundfont = "/usr/share/sounds/sf2/FluidR3_GM.sf2"
fs = FluidSynth(soundfont)
fs.midi_to_audio(midi_file, "melody.wav")

print("Done: melody.wav created.")


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'melody.wav'..
Done: melody.wav created.
